In [143]:
%load_ext autoreload
%autoreload 2
from building_dection import *

import os
import rasterio as rio
import numpy as np
import json
import pyproj
from osgeo import gdal
import subprocess
import zipfile

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, LTAError, LTATriggered
from datetime import date

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
sn7_path = "/scratch/toml/sn7"
train_path = os.path.join(sn7_path, "SN7_buildings_train/train")
sentinel_path = os.path.join(sn7_path, "sentinel2")

In [145]:
labels = get_labels(train_path)[:3]

Label file /scratch/toml/sn7/SN7_buildings_train/train/L15-0358E-1220N_1433_3310_13/labels/global_monthly_2018_01_mosaic_L15-0358E-1220N_1433_3310_13_Buildings.geojson not found
Label file /scratch/toml/sn7/SN7_buildings_train/train/L15-0577E-1243N_2309_3217_13/labels/global_monthly_2018_01_mosaic_L15-0577E-1243N_2309_3217_13_Buildings.geojson not found
Label file /scratch/toml/sn7/SN7_buildings_train/train/L15-0614E-0946N_2459_4406_13/labels/global_monthly_2018_01_mosaic_L15-0614E-0946N_2459_4406_13_Buildings.geojson not found
Label file /scratch/toml/sn7/SN7_buildings_train/train/L15-1200E-0847N_4802_4803_13/labels/global_monthly_2018_01_mosaic_L15-1200E-0847N_4802_4803_13_Buildings.geojson not found
Label file /scratch/toml/sn7/SN7_buildings_train/train/L15-1389E-1284N_5557_3054_13/labels/global_monthly_2018_01_mosaic_L15-1389E-1284N_5557_3054_13_Buildings.geojson not found
Label file /scratch/toml/sn7/SN7_buildings_train/train/L15-1439E-1134N_5759_3655_13/labels/global_monthly_2018

In [146]:
products = get_sentinel2_product_ids(labels[:2])

L15-0331E-1257N_1327_3160_13
Found product
L15-0357E-1223N_1429_3296_13
Found product


In [147]:
downloaded_prods = download_sentinel2_products(products, sentinel_path)

Product a514a273-f5a5-41b2-aac8-7a714f558038 already downloaded
Product bcd5474b-ca6d-4913-b4ab-d525cf075cf7 already downloaded


In [148]:
locs = {}
for prod in downloaded_prods:
    locs[prod['label']] = f"{prod['title']}.SAFE"

modifications = [
    {
        "flag": "SEN2COR_MODIFY_LUT_AEROSOL",
        "value": "MARITIME",
        "info": "Look Up Table Aerosol set to MARITIME",
        "name": "LUT_AEROSOL_MARITIME",
    },
    {
        "flag": "SEN2COR_MODIFY_LUT_SEASON",
        "value": "WINTER",
        "info": "Look Up Table Mid Latitude Season set to WINTER",
        "name": "LUT_SEASON_WINTER",
    },
]


In [149]:
%autoreload 2
from l2a_analysis import L2A_Analysis

l2a_analysis = L2A_Analysis(report_name="space_net_7_01_2020", base_input_dir="/scratch/toml/sn7/sentinel2", base_output_dir="/scratch/toml/sn7/reports")
l2a_analysis.set_locations(locs)
l2a_analysis.set_modifications(modifications)


In [150]:
l2a_analysis.run_sen2cor()

Running sen2cor for L15-0331E-1257N_1327_3160_13: Reference
Running command:  /scratch/toml/Sen2Cor-02.11.00-Linux64/bin/L2A_Process /scratch/toml/sn7/sentinel2/L15-0331E-1257N_1327_3160_13/S2B_MSIL1C_20230911T184939_N0509_R113_T10SFH_20230911T223318.SAFE --resolution 60 --GIP_L2A /home/toml/sen2cor/2.11/cfg/L2A_GIPP_custom.xml --output_dir /scratch/toml/sn7/reports/space_net_7_01_2020/L15-0331E-1257N_1327_3160_13/reference
Product versions > 14.9 are not implemented yet.
Version 14.9 will be used by default, but warnings during validation might occur.

Cloud heights processed in 0:00:09.591338 seconds
running old post processing
Running modify_SCL in postprocessing step
Finished modify_SCL
Setting log level to DEBUG and running atmospheric correction
	SCL
	SCL
	SCL
	B12
	B8A
	B04
	SCL
	B04
	SCL
	VIS
	B04
	SCL
	SCL
	B04
	B03
	B8A
	SCL
	SCL
	B02
	B04
	B01
	SCL
	SCL
	VIS
	AOT
	B8A
	B09
	SCL
	B09
	B8A
	B02
	B09
	B8A
	B02
	WVP
	VIS
	WVP
	SCL
	WVP
	SCL
	VIS
	B01
	B02
	B03
	B04
	B05
	B06
	B0

In [151]:
l2a_analysis.read_l2a_data()

Added band B01 to stack
Added band B02 to stack
Added band B03 to stack
Added band B04 to stack
Added band B05 to stack
Added band B06 to stack
Added band B07 to stack
Added band B8A to stack
Added band B09 to stack
Added band B11 to stack
Added band B12 to stack
Added band SCL to stack
Added band AOT to stack
Added band WVP to stack
Added band TCI to stack
Added band B01 to stack
Added band B02 to stack
Added band B03 to stack
Added band B04 to stack
Added band B05 to stack
Added band B06 to stack
Added band B07 to stack
Added band B8A to stack
Added band B09 to stack
Added band B11 to stack
Added band B12 to stack
Added band SCL to stack
Added band AOT to stack
Added band WVP to stack
Added band TCI to stack
Added band B01 to stack
Added band B02 to stack
Added band B03 to stack
Added band B04 to stack
Added band B05 to stack
Added band B06 to stack
Added band B07 to stack
Added band B8A to stack
Added band B09 to stack
Added band B11 to stack
Added band B12 to stack
Added band SCL t

In [152]:
# rasterize labels, crop to extent of sentinel products, save as tif and orgainize in train folder

train_base_dir = "/scratch/toml/sn7/train"

for prod in downloaded_prods:
    label = prod["label"]
    label_dir = os.path.join(train_base_dir, label)
    if os.path.exists(label_dir) == False:
        os.mkdir(label_dir)
        os.mkdir(os.path.join(label_dir, "images"))
        
        # reproject and rasterize building labels
        reference_band = l2a_analysis.reference_bands[label]['B02']
        with rio.open(reference_band.jp2_path) as src:
            extent = src.bounds
            tgt_crs = src.crs.to_string()

        # all building labels are in epsg:4326
        src_crs = 'EPSG:4326'
        
        label_geojsons = os.listdir(f"{train_path}/{label}/labels")
        building_labels_geojson = label_geojsons[0]
        original_labels = f"{train_path}/{label}/labels/{building_labels_geojson}"
        reprojected_labels = f"{label_dir}/{building_labels_geojson}"
        rasterized_labels = f"{reprojected_labels.split('.')[0]}.tif"
        cropped_labels = f"{rasterized_labels.split('.')[0]}_cropped.tif"
        
        from building_dection import reproject_geojson, rasterize_labels, crop_to_extent
        reproject_geojson(original_labels, src_crs, tgt_crs, out_fn=reprojected_labels)
        rasterize_labels(reprojected_labels, rasterized_labels, extent)
        crop_to_extent(original_labels, rasterized_labels, cropped_labels)
        # rasterize labels

        ref_image_dir = os.path.join(label_dir, "images","ref") 
        os.mkdir(ref_image_dir)
        for band_name in l2a_analysis.reference_bands[label].bands:
            band = l2a_analysis.reference_bands[label][band_name]
            src_path = band.jp2_path
            tgt_path = f"{ref_image_dir}/{band.name}.tif"
            crop_to_extent(original_labels,src_path, tgt_path)
        for mod in l2a_analysis.modifications:
            mod_image_dir = os.path.join(label_dir, "images", mod["name"])
            os.mkdir(mod_image_dir)
            for band_name in l2a_analysis.modified_bands[label][mod["name"]].bands:
                band = l2a_analysis.modified_bands[label][mod["name"]][band_name]
                src_path = band.jp2_path
                tgt_path = f"{mod_image_dir}/{band.name}.tif"
                crop_to_extent(original_labels,src_path, tgt_path)

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
